In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the first page to scrape
url = 'https://growershouse.com/indoor-led-grow-lights?product_list_limit=96'
driver.get(url)

# Wait for 5 seconds to allow manual closing of the popup
print("Please close the popup within the next 5 seconds...")
time.sleep(5)

# Open a CSV file to write product links
with open('product_links.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Link'])  # Write header

    # Function to extract product links
    def extract_product_links():
        products = driver.find_elements(By.CLASS_NAME, 'item.product.product-item')
        for product in products:
            try:
                # Extract the link
                link = product.find_element(By.TAG_NAME, 'a').get_attribute('href')
                writer.writerow([link])  # Write link to CSV
            except Exception as e:
                print(f'Error extracting link: {e}')

    # Function to handle pagination by directly navigating to the next page
    def scrape_all_pages():
        page_number = 1
        while True:
            print(f"Scraping page {page_number}")
            extract_product_links()

            # Try to find the "Next" button
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '.pages-item-next a.link.next'))
                )
                # If found, click it
                next_button.click()
                time.sleep(3)  # Wait for the new page to load
                page_number += 1
            except Exception as e:
                print(f'No more pages or error: {e}')
                break

        # Navigate to the second page directly if there are more pages
        driver.get('https://growershouse.com/indoor-led-grow-lights?p=2&product_list_limit=96')
        time.sleep(3)  # Wait for the new page to load
        print("Navigated to the second page")

        # Continue scraping from the second page
        extract_product_links()

# Scrape all pages
scrape_all_pages()

# Close the browser
driver.quit()
https://growershouse.com/indoor-led-grow-lights?p=2&product_list_limit=96

Please close the popup within the next 5 seconds...
Scraping page 1
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O operation on closed file.
Error extracting link: I/O oper

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00BB8923+23283]
	(No symbol) [0x00B7E934]
	(No symbol) [0x00AB0733]
	(No symbol) [0x00A8D2E3]
	(No symbol) [0x00B1A64F]
	(No symbol) [0x00B2C686]
	(No symbol) [0x00B141B6]
	(No symbol) [0x00AE8017]
	(No symbol) [0x00AE890D]
	GetHandleVerifier [0x00CAA5F3+1013699]
	GetHandleVerifier [0x00CB3E4C+1052700]
	GetHandleVerifier [0x00CAD4B4+1025668]
	GetHandleVerifier [0x00BDEA2B+179195]
	(No symbol) [0x00B86833]
	(No symbol) [0x00B83198]
	(No symbol) [0x00B83337]
	(No symbol) [0x00B7B4BE]
	BaseThreadInitThunk [0x76377BA9+25]
	RtlInitializeExceptionChain [0x7707C10B+107]
	RtlClearBits [0x7707C08F+191]


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import time

# Set up the Chrome driver with rotating user-agents
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:47.0) Gecko/20100101 Firefox/47.0",
    # Add more user agents if needed
]

# Randomly select a user-agent for each session
chrome_options = Options()
chrome_options.add_argument(f"user-agent={random.choice(user_agents)}")

driver = webdriver.Chrome(options=chrome_options)

# Read the CSV file with product links
product_links_df = pd.read_csv('product_links.csv')
product_urls = product_links_df['Product Link'].tolist()

# Step 1: Collect all possible specification names
all_spec_names = set()

def random_pause():
    pause_time = random.uniform(2, 10)  # Random pause between 2 to 10 seconds
    time.sleep(pause_time)

def collect_spec_names(url):
    driver.get(url)
    random_pause()  # Pause before scraping
    time.sleep(3)  # Wait for the page to load

    specs_section = driver.find_elements(By.CSS_SELECTOR, 'div.additional-attributes-wrapper table tr')
    for row in specs_section:
        spec_name = row.find_element(By.TAG_NAME, 'th').text
        all_spec_names.add(spec_name)

for url in product_urls:
    collect_spec_names(url)

# Convert spec names to a list for consistent ordering
all_spec_names = list(all_spec_names)

# Initialize an empty DataFrame with all possible columns
df = pd.DataFrame(columns=['Product Name', 'Product Description', 'Price'] + all_spec_names)

# Step 2: Scrape the product details and append to the DataFrame
def scrape_product_details(url):
    driver.get(url)
    random_pause()  # Pause before scraping
    time.sleep(3)  # Wait for the page to load

    product_data = {name: '' for name in all_spec_names}  # Initialize with all possible spec names

    # Extract the product name
    product_name = driver.find_element(By.CLASS_NAME, 'page-title').text
    product_data['Product Name'] = product_name

    # Extract the product description
    product_description = driver.find_element(By.CLASS_NAME, 'product.attribute.overview').text
    product_data['Product Description'] = product_description

    # Extract specifications from the "additional-attributes-wrapper" class
    specs_section = driver.find_elements(By.CSS_SELECTOR, 'div.additional-attributes-wrapper table tr')
    for row in specs_section:
        spec_name = row.find_element(By.TAG_NAME, 'th').text
        spec_value = row.find_element(By.TAG_NAME, 'td').text
        product_data[spec_name] = spec_value

    # Extract the price, if available
    try:
        price = driver.find_element(By.CLASS_NAME, 'price').text
        product_data['Price'] = price
    except:
        product_data['Price'] = 'Not listed'

    return product_data

# Scrape each product and add to the DataFrame
for url in product_urls:
    product_details = scrape_product_details(url)
    df = df.append(product_details, ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('dynamic_product_details.csv', index=False)

# Close the browser
driver.quit()
